In [2]:
# generate_data.py
import numpy as np
import random

def generate_branch_history_complex(num_iterations=500, noise_level=0.05, history_depth=5):
    """Generates a more complex history based on combined patterns and noise."""
    if history_depth < 3:
        raise ValueError("history_depth must be at least 3 for this logic")

    history = []
    # Simulate initial random history
    history.extend(np.random.randint(0, 2, size=history_depth).tolist())

    print("Generating complex history...")
    print("="*20)
    print("Iter | Modulo7 | Hist[",history_depth-1,"]==0 | XOR | Noise Flip | Final Outcome")
    print("-"*70)


    for i in range(history_depth, num_iterations):
        # Pattern 1: Periodic but less frequent
        periodic_pattern = (i % 7 == 0) # True if multiple of 7

        # Pattern 2: Correlation with a specific past bit (e.g., 5 steps ago)
        # Ensure we have enough history before accessing history[-history_depth]
        correlation_pattern = (history[i - history_depth] == 0) # True if the bit 'history_depth' steps ago was NT (0)

        # Combine patterns using XOR: Outcome is 1 if *exactly one* pattern is true
        deterministic_outcome = 1 if periodic_pattern ^ correlation_pattern else 0

        # Introduce Noise: Small chance to flip the outcome
        noise_flip = False
        final_outcome = deterministic_outcome
        if random.random() < noise_level:
            final_outcome = 1 - deterministic_outcome # Flip the outcome
            noise_flip = True

        history.append(final_outcome)

        # Optional: Print for debugging the first few steps
        if i < history_depth + 15 :
             print(f"{i:4d} | {int(periodic_pattern):^9d} | {int(correlation_pattern):^10d} | {deterministic_outcome:^3d} | {str(noise_flip):^10s} | {final_outcome:^13d}")


    print("-"*70)
    print("... History generation complete.")
    return np.array(history)

def create_windows(data, history_window_size):
    """Creates sliding windows for sequence prediction."""
    X, y = [], []
    if len(data) <= history_window_size:
        print(f"Error: Data length ({len(data)}) must be greater than history_window_size ({history_window_size})")
        return np.array(X), np.array(y)

    for i in range(history_window_size, len(data)):
        X.append(data[i-history_window_size:i])
        y.append(data[i])
    return np.array(X), np.array(y)

if __name__ == "__main__":
    HISTORY_WINDOW_SIZE = 16 # How many past branches BranchNet looks at
    NUM_SAMPLES = 1000     # Increase samples for better training/testing
    NOISE = 0.00           # ~8% chance of random outcome flip
    HISTORY_DEPTH_CORR = 5 # Look 5 steps back for correlation pattern

    print(f"Generating {NUM_SAMPLES} complex branch outcomes (Noise={NOISE*100:.1f}%, Correlation Depth={HISTORY_DEPTH_CORR})...")
    full_history = generate_branch_history_complex(NUM_SAMPLES, NOISE, HISTORY_DEPTH_CORR)

    print(f"\nCreating training windows (size {HISTORY_WINDOW_SIZE})...")
    X, y = create_windows(full_history, HISTORY_WINDOW_SIZE)

    if X.size == 0:
        print("Failed to create windows. Exiting.")
    else:
        # Reshape X for CNN/LSTM input: (num_samples, timesteps, features)
        X = X.reshape(X.shape[0], X.shape[1], 1)

        print(f"Generated X shape: {X.shape}")
        print(f"Generated y shape: {y.shape}")

        # Save the generated data
        np.savez('branchnet_training_data.npz', X=X, y=y, full_history=full_history)
        print("Data saved to branchnet_training_data.npz")

Generating 1000 complex branch outcomes (Noise=0.0%, Correlation Depth=5)...
Generating complex history...
Iter | Modulo7 | Hist[ 4 ]==0 | XOR | Noise Flip | Final Outcome
----------------------------------------------------------------------
   5 |     0     |     1      |  1  |   False    |       1      
   6 |     0     |     0      |  0  |   False    |       0      
   7 |     1     |     1      |  0  |   False    |       0      
   8 |     0     |     1      |  1  |   False    |       1      
   9 |     0     |     1      |  1  |   False    |       1      
  10 |     0     |     0      |  0  |   False    |       0      
  11 |     0     |     1      |  1  |   False    |       1      
  12 |     0     |     1      |  1  |   False    |       1      
  13 |     0     |     0      |  0  |   False    |       0      
  14 |     1     |     0      |  1  |   False    |       1      
  15 |     0     |     1      |  1  |   False    |       1      
  16 |     0     |     0      |  0  |   Fa